In [1]:
import torch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn as nn
import torch.nn.functional as F

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
print("RUN CELL")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/das-cnn-cars/data.pt
RUN CELL


In [2]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, imgs, counts, labels, ids, transforms = lambda x: x):
      self.imgs = imgs
      self.labels = labels
      self.counts = counts
      self.labels = labels
      self.transforms = transforms
      self.ids = ids

    def __getitem__(self, i, return_id = False):
      if return_id:
        return (self.transforms(self.imgs[i].unsqueeze(0)), self.labels[i], self.counts[i], self.ids[i])
      return (self.transforms(self.imgs[i].unsqueeze(0)), self.labels[i], self.counts[i])

    def __len__(self):
      return len(self.imgs)

In [3]:
class SimpleVehicleNet(nn.Module):
    def __init__(self, num_bins=8, num_count_classes = 7):  # num_bins = number of vehicle count buckets
        super(SimpleVehicleNet, self).__init__()
        self.num_count_classes = num_count_classes
        # Input: [B, 1, 585, 130]
        self.backbone = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),  # [B, 16, 585, 130]
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 16, 292, 65]

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),  # [B, 32, 292, 65]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 32, 146, 32]

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # [B, 64, 146, 32]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),  # [B, 64, 1, 1]
        )

        self.flatten = nn.Flatten()  # → [B, 64]

        # Regression head (predict scalar count)
        self.count_head = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_count_classes)
        )

        # Distribution head (predict vehicle count histogram)
        self.dist_head = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, num_bins)  # Output: raw logits → apply log_softmax
        )
        self.mse = nn.MSELoss()
        self.mae = nn.L1Loss()
        self.cross_entropy = nn.CrossEntropyLoss()
        self.kl = nn.KLDivLoss(reduction = "batchmean")
        self.lambda_count = 10
        self.lambda_kl = 1
        self.lambda_mse = 1
        self.freq = torch.Tensor([3.8822e-02, 5.4998e-03, 8.6703e-01, 4.8528e-03, 0.0000e+00, 0.0000e+00,
        3.2352e-04, 8.3468e-02])

    def masked_mse(self, y_pred, y_true):
        # Create a mask where target is non-zero
        mask = (y_true != 0).float()
        
        # Compute squared error only where mask == 1
        loss = (mask * (y_pred - y_true) ** 2)
        
        # Avoid division by zero: normalize by number of non-zero elements
        return loss.sum() / (mask.sum() + 1e-8)

    def weighted_mse(self, y_pred, y_true):
        weights = 1.0 / (self.freq + 1e-6)
        weights = weights / weights.sum()
        weights = weights.to(y_pred.device)
        loss = weights * (y_pred - y_true) ** 2
        return loss.mean()
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.flatten(x)
        count = self.count_head(x)                      # shape: [B, 1]
        dist_logits = self.dist_head(x)                # shape: [B, num_bins]
        dist_output = F.softmax(dist_logits, dim=1)
        return count, dist_output

    def loss(self, count_out, hist_out, count_label, hist_label):
        label = F.one_hot(count_label, num_classes = self.num_count_classes).float()
        #print(count_out.shape, label.shape)
        #print(count_out.dtype, label.dtype)
        count_loss = self.cross_entropy(count_out, label)
        row_sums = hist_label.sum(dim=1, keepdim=True)
        normalized = hist_label / row_sums
        #hist_loss = self.weighted_mse(hist_out, hist_label)
        hist_loss = self.cross_entropy(hist_out, normalized)
        return self.lambda_count * count_loss + hist_loss

In [4]:
class CountCNN(nn.Module):
    def __init__(self, classes = 7):  # num_bins = number of vehicle count buckets
        super(CountCNN, self).__init__()

        # Input: [B, 1, 585, 130]
        self.backbone = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),  # [B, 16, 585, 130]
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 16, 292, 65]

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),  # [B, 32, 292, 65]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # [B, 32, 146, 32]

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # [B, 64, 146, 32]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),  # [B, 64, 1, 1]
        )

        self.flatten = nn.Flatten()  # → [B, 64]

        # Regression head (predict scalar count)
        self.count_head = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, classes)
        )
        
        self.mse = nn.MSELoss()
        self.mae = nn.L1Loss()
        self.cross_entropy = nn.CrossEntropyLoss()
        self.freq = torch.Tensor([3.8822e-02, 5.4998e-03, 8.6703e-01, 4.8528e-03, 0.0000e+00, 0.0000e+00,
        3.2352e-04, 8.3468e-02])
        self.classes = classes
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.flatten(x)
        count = self.count_head(x)                      # shape: [B, 1]
        return count

    def loss(self, count_out, count_label):
        label = F.one_hot(count_label, num_classes = self.classes).float()
        count_loss = self.cross_entropy(count_out, count_label)
        return count_loss

In [19]:
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F

def append_dropout(model, rate=0.2):
    for name, module in model.named_children():
        if len(list(module.children())) > 0:
            append_dropout(module)
        if isinstance(module, nn.ReLU):
            new = nn.Sequential(module, nn.Dropout2d(p=rate))
            setattr(model, name, new)
            
class VehicleCounterNet(nn.Module):
    def __init__(self, num_classes=8, count_classes = 7):
        super(VehicleCounterNet, self).__init__()

        # Load pretrained ResNet18 and modify
        self.backbone = models.resnet18()
        #append_dropout(self.backbone, rate = 0.1)
        # Modify input conv layer to take 1 channel (grayscale)
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Replace the final FC layer with identity so we can define our own heads
        num_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()

        # Output head 1: Regress or classify number of vehicles
        self.vehicle_count = nn.Sequential(nn.Linear(num_features, count_classes))

        # Output head 2: Predict histogram (e.g., soft count distribution across possible values)
        self.histogram_head = nn.Sequential(nn.Linear(num_features, num_classes))
        self.cross_entropy = nn.CrossEntropyLoss()
        self.freq = torch.Tensor([3.8822e-02, 5.4998e-03, 8.6703e-01, 4.8528e-03, 0.0000e+00, 0.0000e+00,
        3.2352e-04, 8.3468e-02])
        self.weighted_nll = nn.NLLLoss(weight = self.freq)
        #lambda P, Q: -torch.sum(self.freq*P*torch.log(Q + 1e-9))
        self.mse = nn.MSELoss()
        self.kl = nn.KLDivLoss(reduction = "batchmean")
        self.lambda_count = 10
        self.lambda_kl = 1
        self.lambda_mse = 1
        self.count_classes = count_classes

    def forward(self, x):
        features = self.backbone(x)
        count_output = self.vehicle_count(features)
        histogram_output = F.softmax(self.histogram_head(features), dim = 1)
        return count_output, histogram_output

    def masked_mse(self, y_pred, y_true):
        # Create a mask where target is non-zero
        mask = (y_true != 0).float()
        
        # Compute squared error only where mask == 1
        loss = (mask * (y_pred - y_true) ** 2)
        
        # Avoid division by zero: normalize by number of non-zero elements
        return loss.sum() / (mask.sum() + 1e-8)

    def weighted_soft_nll_loss(self, log_probs, soft_targets):
        """
        log_probs: Tensor of shape [batch_size, num_classes], output of log_softmax
        soft_targets: Tensor of shape [batch_size, num_classes], target distributions
        class_weights: Tensor of shape [num_classes], weight per class
        """
        # Expand class_weights to match soft_targets shape
        weights = self.freq.unsqueeze(0).to(log_probs.device)  # [1, num_classes]
        
        # Apply weights to soft_targets
        weighted_targets = soft_targets * weights  # [batch_size, num_classes]
        
        # Compute element-wise product: -weighted_targets * log_probs
        loss = -torch.sum(weighted_targets * log_probs, dim=1)  # [batch_size]
        
        return loss.mean()  # or use .sum() if preferred



    def loss(self, count_out, hist_out, count_label, hist_label):
        count_label = F.one_hot(count_label, self.count_classes).float()
        count_loss = self.cross_entropy(count_out, count_label)
        target = hist_label + 1e-8
        target_probs = (target) / target.sum(dim=1, keepdim=True)
        expected_hist_loss = self.weighted_soft_nll_loss(torch.log(hist_out), target_probs)
        total_loss = (
          self.lambda_count * count_loss +
          self.lambda_mse * expected_hist_loss
        )
        
        return total_loss

In [6]:
import os
from datetime import datetime

class Logger:
  def __init__(self, log_dir, filename='train_log.txt'):
      os.makedirs(log_dir, exist_ok=True)
      timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
      self.log_dir = log_dir
      self.log_path = os.path.join(log_dir, f'{timestamp}_{filename}')

      with open(self.log_path, 'w') as f:
          f.write(f"Logging started: {timestamp}\n\n")

  def log(self, message):
      timestamp = datetime.now().strftime('%H:%M:%S')
      full_message = f"[{timestamp}] {message}"
      print(full_message)  # also print to stdout
      with open(self.log_path, 'a') as f:
          f.write(full_message + '\n')

  def log_metrics(self, epoch, train_loss=None, val_loss=None, **kwargs):
      msg = f"Epoch {epoch} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}"
      for k, v in kwargs.items():
          msg += f" | {k}: {v:.4f}" if isinstance(v, float) else f" | {k}: {v}"
      self.log(msg)

  def save_checkpoint(self, path, model, optimizer, epoch, best_val_loss):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'best_val_loss': best_val_loss
    }
    torch.save(checkpoint, os.path.join(self.log_dir, path))


In [7]:

data = torch.load("/kaggle/input/das-cnn-cars/data.pt")

/tmp/ipykernel_31/3726580803.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("/kaggle/input/das-cnn-cars/data.pt")


In [8]:
print(data)

{'imgs': tensor([[[-76.5721, -77.1576, -81.6660,  ..., -83.9075, -81.3769, -84.3168],
         [-76.3438, -73.9983, -79.6962,  ..., -80.6483, -82.2012, -87.3538],
         [-76.6467, -74.0830, -81.2737,  ..., -81.5403, -83.2910, -88.4794],
         ...,
         [-79.3313, -78.3320, -78.4678,  ..., -81.0519, -80.5077, -88.1365],
         [-85.3054, -79.8374, -75.8968,  ..., -86.9083, -82.3489, -84.4151],
         [-85.2439, -82.8430, -79.4188,  ..., -80.8857, -87.3668, -87.8926]],

        [[-80.6762, -81.7623, -72.8934,  ..., -85.9792, -84.2322, -86.8569],
         [-87.8347, -79.4662, -78.5318,  ..., -81.9379, -86.0406, -86.1646],
         [-86.9451, -79.9672, -75.6647,  ..., -81.5951, -82.0449, -87.3756],
         ...,
         [-83.1090, -84.2905, -83.3142,  ..., -66.5070, -70.2364, -76.1732],
         [-81.8832, -83.0903, -81.6652,  ..., -79.0737, -79.5598, -82.4076],
         [-81.5252, -81.8138, -81.5861,  ..., -80.0465, -84.4012, -88.0107]],

        [[-81.3736, -82.3884, -82.3

In [9]:
import matplotlib.pyplot as plt
counts = data["counts"]
print(torch.unique(counts, return_counts = True))

(tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 17, 18, 19]), tensor([309, 111,  85,  77, 253, 122, 109,  85,  73, 100,  29,  13,  11,   3,
          8,   1,   1,   1]))


In [10]:
filtered_data = {}
filtered_data["imgs"] = []
filtered_data["counts"] = []
filtered_data["labels"] = []
filtered_data["ids"] = []
#filter out when counts <= 6
for i in range(len(counts)):
    if counts[i] <= 6:
        for k in ["imgs", "counts", "labels", "ids"]:
            filtered_data[k].append(data[k][i].tolist())

In [11]:
for k in ["imgs", "counts", "labels", "ids"]:
    filtered_data[k] = torch.tensor(filtered_data[k])
    print(filtered_data[k].shape)

torch.Size([957, 585, 153])
torch.Size([957])
torch.Size([957, 8])
torch.Size([957])


In [12]:
from torch.utils.data import random_split, DataLoader
from torchvision.transforms import v2
images = filtered_data["imgs"]
counts = filtered_data["counts"]
labels = filtered_data["labels"]
ids = filtered_data["ids"]
train_transform = v2.Compose([
    v2.RandomHorizontalFlip(),         # flip image horizontally
    v2.RandomRotation(15),             # rotate by ±15 degrees
    v2.ToTensor(),                     # convert to tensor
    v2.Normalize((0.5, ), (0.5, ))            # normalize
])
dataset = Dataset(images, counts, labels, ids, transforms = v2.Normalize((0.5, ), (0.5, )))
train_ds, test_ds = random_split(dataset, [0.9, 0.1])

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [13]:
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm

In [21]:
model = VehicleCounterNet() #CountCNN()#VehicleCounterNet() #SimpleVehicleNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
logger = Logger("/kaggle/working/182_proj_logs_cross_entropy/")
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
val_loader = DataLoader(test_ds)
# Loss and optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-2)
train_losses = []
val_losses = []
# Training loop
num_epochs = 100
best_val_loss = float('inf')
logger.save_checkpoint("best_model.pth", model, optimizer, 0, best_val_loss)
for epoch in tqdm(range(num_epochs)):
    # --------- Train ---------
    model.train()
    train_loss = 0.0
    train_count_correct = 0
    train_label_correct = 0
    train_total = 0

    for images, labels, counts in train_loader:
        images = images.to(device).float()
        labels = labels.to(device).float()
        counts = counts.to(device).long()
        optimizer.zero_grad()
        count_output, label_output = model(images)
        loss = model.loss(count_output, label_output, counts, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds = torch.argmax(count_output, dim = -1)
        label_preds = preds.unsqueeze(-1) * label_output
        train_count_correct += (preds == counts).sum().item()
        train_label_correct += (torch.round(label_preds).flatten() == labels.flatten()).sum().item()
        train_total += labels.size(0)

    avg_train_loss = train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    train_acc_count = train_count_correct / train_total
    train_acc_label = train_label_correct / (8*train_total)

    # --------- Validate ---------
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_label_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels, counts in val_loader:
            images = images.to(device).float()
            labels = labels.to(device).float()
            counts = counts.to(device).long()#.unsqueeze(1)
            count_output, label_output = model(images)
            loss = model.loss(count_output, label_output, counts, labels)
            val_loss += loss.item()
            preds = torch.argmax(count_output, dim = -1)
            label_preds = preds.unsqueeze(-1) * label_output
            val_correct += (preds == counts.flatten()).sum().item()
            val_label_correct += (torch.round(label_output).flatten() == labels.flatten()).sum().item()
            val_total += labels.size(0)
            
    
    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    val_acc = val_correct / val_total
    val_acc_label = val_label_correct / (8*val_total)
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        logger.save_checkpoint("best_model.pth", model, optimizer, epoch, best_val_loss)

    # --------- Print results ---------
    
    logger.log(f"Epoch {epoch+1}/{num_epochs} "
          f"Train Loss: {avg_train_loss:.2f} | Count Acc: {train_acc_count:.2f} | Label Acc: {train_acc_label:.2f}"
          f"|| Val Loss: {avg_val_loss:.2f} | Count Acc: {val_acc:.2f} | Label Acc: {val_acc_label:.2f}")
    '''
    logger.log(f"Epoch {epoch+1}/{num_epochs} "
          f"Train Loss: {avg_train_loss:.2f} | Count Acc: {train_acc_count:.2f}"
          f"|| Val Loss: {avg_val_loss:.2f} | Count Acc: {val_acc:.2f}")   
    '''

  1%|          | 1/100 [00:05<09:48,  5.95s/it]

[22:44:59] Epoch 1/100 Train Loss: 18.64 | Count Acc: 0.38 | Label Acc: 0.88|| Val Loss: 19.60 | Count Acc: 0.34 | Label Acc: 0.88


  2%|▏         | 2/100 [00:11<09:42,  5.95s/it]

[22:45:05] Epoch 2/100 Train Loss: 15.37 | Count Acc: 0.44 | Label Acc: 0.90|| Val Loss: 16.64 | Count Acc: 0.37 | Label Acc: 0.88


  3%|▎         | 3/100 [00:17<09:22,  5.80s/it]

...

[22:54:08] Epoch 98/100 Train Loss: 0.07 | Count Acc: 1.00 | Label Acc: 0.93|| Val Loss: 47.53 | Count Acc: 0.43 | Label Acc: 0.88


 99%|█████████▉| 99/100 [09:21<00:05,  5.65s/it]

[22:54:14] Epoch 99/100 Train Loss: 7.69 | Count Acc: 0.81 | Label Acc: 0.91|| Val Loss: 22.08 | Count Acc: 0.39 | Label Acc: 0.88


100%|██████████| 100/100 [09:27<00:00,  5.67s/it]

[22:54:20] Epoch 100/100 Train Loss: 1.10 | Count Acc: 0.97 | Label Acc: 0.93|| Val Loss: 35.72 | Count Acc: 0.42 | Label Acc: 0.88


In [15]:
 with torch.no_grad():
    for images, labels, counts in val_loader:
        images = images.to(device).float()
        labels = labels.to(device).float()
        counts = counts.to(device).float().unsqueeze(1)
        count_output, label_output = model(images)
        loss = model.loss(count_output, label_output, counts, labels)
        val_loss += loss.item()
        preds = torch.round(count_output).flatten()
        val_correct += (preds == counts.flatten()).sum().item()
        val_label_correct += (torch.round(label_output).flatten() == labels.flatten()).sum().item()
        val_total += labels.size(0)
        print(label_output)

RuntimeError: one_hot is only applicable to index tensor of type LongTensor.

In [ ]:
freq = filtered_data["labels"].sum(dim = 0)
freq = freq / freq.sum()
print(freq)